In [ ]:
# Genre & Style Comment Generator — Reusable Template

# This notebook automates the process of tagging music tracks with one-line genre and style comments
# using OpenAI's GPT model. It reads from a CSV, sends track data to GPT, and writes the output back to a file.

# --- IMPORT LIBRARIES --- #
# These import statements bring in all the external tools this notebook needs to function.
# pandas is used for handling and editing tabular data (your CSV of tracks).
# OpenAI gives access to the ChatGPT API.
# os is used for file and folder management.
# time is used for creating delays between API calls (to avoid rate limits).
# tenacity adds automatic retry logic for failed API calls.
# logging lets us see detailed information about what's happening when the notebook runs.

import pandas as pd
from openai import OpenAI
import os
import time
from tenacity import retry, wait_fixed, stop_after_attempt, retry_if_exception_type, before_sleep_log
import logging

# --- LOGGING SETUP FOR RETRIES --- #
# This section sets up logging so we can see retry attempts and API activity.
# It will print useful information to the notebook output (e.g. retry messages, OpenAI request responses).
# This helps diagnose issues if the API fails or is slow.

logging.basicConfig(level=logging.INFO)  # sets the logging level to 'INFO' so we see key activity
logger = logging.getLogger(__name__)     # creates a logger object we can use with tenacity

# --- CONFIGURATION SECTION --- #
# These variables define which file to load and where to save output.
# Change INPUT_FILE to point to a different CSV if you want to process another playlist.

INPUT_FILE = "../data/PureElectroGroove.csv"            # input CSV file with track data
OUTPUT_FILE = "../output/PureElectroGroove_GenreTagged.csv"  # output CSV file to save genre/style comments
OPENAI_API_KEY = "sk-..."                        # Replace with your API key

# --- OPENAI CLIENT SETUP --- #
# This line initializes the OpenAI client using your API key.
# It allows the notebook to send requests to ChatGPT.

client = OpenAI(api_key=OPENAI_API_KEY)

# --- LOAD YOUR TRACK DATA --- #
# This reads the input CSV file into a table-like structure called a DataFrame.
# Each row is a track with columns like 'title', 'artist', and (eventually) 'Comment'.

df = pd.read_csv(INPUT_FILE)
df["Comment"] = df.get("Comment", "")  # This line checks if the 'Comment' column exists. If not, it creates it.

# --- GPT PROMPT FUNCTION --- #
# This function prepares a prompt and sends it to GPT to get a genre/style comment.
# It returns a short, DJ-friendly comment about the track.

@retry(
    wait=wait_fixed(5),
    stop=stop_after_attempt(3),
    retry=retry_if_exception_type(Exception),
    before_sleep=before_sleep_log(logger, logging.INFO)  # this line is what triggers visible retry logs in the notebook
)
def generate_genre_comment(title, artist):
    prompt = f"""
    Describe the track for DJs in one line. Include decade and country if possible. Use vivid club-friendly phrasing.
    Avoid repeating genre terms and keep the tone colorful and stylish.
    Format: <Main genre>; <production traits>; <scene/mood/era info>.

    Track title: "{title}"
    Artist: {artist}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a music genre expert and DJ selector."},
            {"role": "user", "content": prompt.strip()}
        ]
    )
    return response.choices[0].message.content.strip()

# --- MAIN PROCESSING LOOP --- #
# This goes through each track in your CSV file.
# If there's no comment yet, it sends the track info to GPT and adds the response.

for i, row in df.iterrows():
    if not row['Comment'] or pd.isna(row['Comment']):
        print(f"🎧 Track {i+1}: {row['title']} by {row['artist']}")
        df.at[i, "Comment"] = generate_genre_comment(row['title'], row['artist'])
        time.sleep(1.5)  # This waits 1.5 seconds after each API call to avoid hitting rate limits.

# --- SAVE YOUR RESULTS --- #
# This saves the updated DataFrame (with comments) back into a CSV file.

os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Done! Comments saved to {OUTPUT_FILE}")
